# Llamav2を試す

# 参考
[Databricksレポジトリ](https://github.com/databricks/databricks-ml-examples/tree/master/llm-models/llamav2)  
[Qiita: MetaのLlama 2をDatabricksで動かしてみる](https://qiita.com/taka_yayoi/items/bc1da42144826da56ab4)

# modelのダウンロード

In [3]:
# Load model to text generation pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# it is suggested to pin the revision commit hash and not change it for reproducibility because the uploader might change the model afterwards; you can find the commmit history of llamav2-7b-chat in https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/commits/main
model = "meta-llama/Llama-2-7b-chat-hf"
revision = "0ede8dd71e923db6258295621d817ca8714516d4"

tokenizer = AutoTokenizer.from_pretrained(model, padding_side="left")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    revision=revision,
    return_full_text=False
)

# Required tokenizer setting for batch inference
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id


c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.01s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


# プロンプトの準備

In [4]:
# Define prompt template, the format below is from: http://fastml.com/how-to-train-your-own-chatgpt-alpaca-style-part-one/

# Prompt templates as follows could guide the model to follow instructions and respond to the input, and empirically it turns out to make Falcon models produce better responses

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


# 関数の作成

In [5]:
# Define parameters to generate text
def gen_text(prompts, use_template=False, **kwargs):
    if use_template:
        full_prompts = [
            PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
            for prompt in prompts
        ]
    else:
        full_prompts = prompts

    if "batch_size" not in kwargs:
        kwargs["batch_size"] = 1
    
    # the default max length is pretty small (20), which would cut the generated output in the middle, so it's necessary to increase the threshold to the complete response
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 2024

    # configure other text generation arguments, see common configurable args here: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    kwargs.update(
        {
            "pad_token_id": tokenizer.eos_token_id,  # Hugging Face sets pad_token_id to eos_token_id by default; setting here to not see redundant message
            "eos_token_id": tokenizer.eos_token_id,
        }
    )

    outputs = pipeline(full_prompts, **kwargs)
    outputs = [out[0]["generated_text"] for out in outputs]

    return outputs


# 単一の入力で文書生成。

In [4]:
results = gen_text(["What is a large language model?"])
print(results[0])

c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\transformers\generation\utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



A large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. These models are designed to learn the patterns and structures of language by exposure to a wide range of texts, and can be used for a variety of applications such as language translation, text summarization, and language generation.

Some examples of large language models include:

1. BERT (Bidirectional Encoder Representations from Transformers): Developed by Google in 2018, BERT is a powerful language model that has achieved state-of-the-art results on a wide range of natural language processing (NLP) tasks. BERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence.
2. RoBERTa (Robustly Optimized BERT Pretraining Approach): Developed in 2019, RoBERTa is a variant of BERT that was specifically designed for text classification tasks. R

# 日本語は？

In [6]:
results = gen_text(["大規模言語モデルとは？"])
print(results[0])



大規模言語モデル（Big Language Model、BLM）とは、自然言語processing（NLP）における一種の人工知能（AI）で、大規模なデータセットを用いて、自然言語の表現を処理することができるように設計されている。

BLMは、通常、大量の文書やデータセットを用いて、言語モデルを学習することができる。これらのデータセットには、単語や文、文章などの自然言語の表現が含まれている。BLMは、これらのデータセットを用いて、自然言語の表現を処理するための特殊な neural network を学習する。

BLMは、以下のような役割を果たすことができる。

1. 文本生成：BLMを使用すると、新しい文本を生成することができます。これには、単語や文、文章などの自然言語の表現が含まれています。
2. 文本理解：BLMを使用すると、既存の文本を理解することができます。これには、単語や文、文章などの自然言語の表現が含まれています。
3. 単語の定義：BLMを使用すると、単語の定義を学習することができます。これには、単語の意味や、単語が使用される文脈などが含まれています。
4. 文法分析：BLMを使


In [8]:
results = gen_text(["""桃太郎の物語を教えてください。                   
"""])
print(results[0])


桃太郎は、日本の民話の登場人物であり、「桃太郎」という名前を持つ。彼は、桃の木の下で生まれ、桃太郎と名付けられた。彼は、妖精の姉妹たちとともに暮らし、桃の木の下で暮らしていた。

ある日、桃太郎は、妖精たちとともに桃の木の下で遊んでいた。彼らは、桃の木の下で遊んでいるうちに、桃太郎が妖精たちに対して、「桃太郎の物語」を語り始めた。

彼らは、桃太郎が妖精たちに語った物語を、「桃太郎の物語」と呼んだ。彼らは、桃太郎の物語を聞いて、彼の勇敢さと優しさに感動した。

桃太郎の物語は、妖精たちにとっては、大切な物語であり、彼らの心に深く刻まれた。彼らは、桃太郎の物語を、彼の勇敢さと優しさを讃えるために、彼の名前を呼び続けた。

以上が、桃太郎の物語の要点です。彼の物語は、勇敢さと優しさを讃えるために、妖精たちにとっては、大切な物語であり、彼らの心に深く刻まれた。


## 桃太郎の物語を知らないようだった。

# 桃太郎の物語でfine-tuningを行う。

# その前にDollyデータセットでfine-tuningを行う。

In [6]:
!python -m pip index versions bitsandbytes --index-url=https://jllllll.github.io/bitsandbytes-windows-webui

bitsandbytes (0.41.1)
Available versions: 0.41.1, 0.41.0, 0.40.2, 0.40.1.post1, 0.40.0.post4, 0.40.0, 0.39.1, 0.39.0, 0.38.1, 0.37.2, 0.35.4, 0.35.0
  INSTALLED: 0.41.1
  LATEST:    0.41.1


In [7]:
!python -m pip install bitsandbytes==0.39.1 --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui
     ---------------------------------------- 0.0/137.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/137.9 MB 1.7 MB/s eta 0:01:24
     ---------------------------------------- 0.6/137.9 MB 7.4 MB/s eta 0:00:19
     --------------------------------------- 1.6/137.9 MB 11.3 MB/s eta 0:00:13
      -------------------------------------- 3.2/137.9 MB 16.9 MB/s eta 0:00:08
     - ------------------------------------- 5.3/137.9 MB 22.6 MB/s eta 0:00:06
     -- ------------------------------------ 7.5/137.9 MB 26.8 MB/s eta 0:00:05
     -- ------------------------------------ 9.7/137.9 MB 29.5 MB/s eta 0:00:05
     --- ---------------------------------- 11.9/137.9 MB 43.5 MB/s eta 0:00:03
     --- ---------------------------------- 14.0/137.9 MB 46.7 MB/s eta 0:00:03
     ---- --------------------------------- 16.3/137.9 MB 46.7 MB/s eta 0:00:03
     ----- ------------------

In [1]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Fine tune llama-2-7b-hf with QLORA
# MAGIC
# MAGIC [Llama 2](https://huggingface.co/meta-llama) is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 70 billion parameters. It is trained with 2T tokens and supports context length window upto 4K tokens. [Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) is the 7B pretrained model, converted for the Hugging Face Transformers format.
# MAGIC
# MAGIC This is to fine-tune [llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) models on the [databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k) dataset.
# MAGIC
# MAGIC Environment for this notebook:
# MAGIC - Runtime: 13.2 GPU ML Runtime
# MAGIC - Instance: `g5.8xlarge` on AWS, `Standard_NV36ads_A10_v5` on Azure
# MAGIC
# MAGIC We leverage the PEFT library from Hugging Face, as well as QLoRA for more memory efficient finetuning.

# COMMAND ----------

from huggingface_hub import notebook_login

# Login to Huggingface to get access to the model
notebook_login()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Install required packages
# MAGIC
# MAGIC Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

# COMMAND ----------

# MAGIC %pip install git+https://github.com/huggingface/peft.git
# MAGIC %pip install datasets==2.12.0 bitsandbytes==0.40.1 einops==0.6.1 trl==0.4.7
# MAGIC %pip install torch==2.0.1 accelerate==0.21.0 transformers==4.31.0

# COMMAND ----------

# MAGIC %md
# MAGIC ## Dataset
# MAGIC
# MAGIC We will use the [databricks-dolly-15k ](https://huggingface.co/datasets/databricks/databricks-dolly-15k) dataset.

# COMMAND ----------

from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_name, split="train")

# COMMAND ----------

INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

def apply_prompt_template(examples):
  instruction = examples["instruction"]
  response = examples["response"]
  context = examples.get("context")

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
  else:
    full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)

# COMMAND ----------

dataset["text"][0]

# COMMAND ----------

# MAGIC %md
# MAGIC ## Loading the model
# MAGIC
# MAGIC In this section we will load the [LLaMAV2](), quantize it in 4bit and attach LoRA adapters on it.

# COMMAND ----------

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"
revision = "351b2c357c69b4779bde72c0e7f7da639443d904"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    revision=revision,
    trust_remote_code=True,
)
model.config.use_cache = False

# COMMAND ----------

# MAGIC %md
# MAGIC Load the configuration file in order to create the LoRA model. 
# MAGIC
# MAGIC According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance. Therefore we will add `dense`, `dense_h_to_4_h` and `dense_4h_to_h` layers in the target modules in addition to the mixed query key value layer.

# COMMAND ----------

from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Loading the trainer

# COMMAND ----------

# MAGIC %md
# MAGIC Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

# COMMAND ----------

from transformers import TrainingArguments

output_dir = "/local_disk0/results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 500
logging_steps = 100
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)

# COMMAND ----------

# MAGIC %md
# MAGIC Then finally pass everthing to the trainer

# COMMAND ----------

from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# COMMAND ----------

# MAGIC %md
# MAGIC We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

# COMMAND ----------

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Train the model

# COMMAND ----------

# MAGIC %md
# MAGIC Now let's train the model! Simply call `trainer.train()`

# COMMAND ----------

trainer.train()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Save the LORA model

# COMMAND ----------

trainer.save_model("/local_disk0/llamav2-7b-lora-fine-tune")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Log the fine tuned model to MLFlow

# COMMAND ----------

import torch
from peft import PeftModel, PeftConfig

peft_model_id = "/local_disk0/llamav2-7b-lora-fine-tune"
config = PeftConfig.from_pretrained(peft_model_id)

from huggingface_hub import snapshot_download
# Download the Llama-2-7b-hf model snapshot from huggingface
snapshot_location = snapshot_download(repo_id=config.base_model_name_or_path)


# COMMAND ----------

import mlflow
class LLAMAQLORA(mlflow.pyfunc.PythonModel):
  def load_context(self, context):
    self.tokenizer = AutoTokenizer.from_pretrained(context.artifacts['repository'])
    self.tokenizer.pad_token = tokenizer.eos_token
    config = PeftConfig.from_pretrained(context.artifacts['lora'])
    base_model = AutoModelForCausalLM.from_pretrained(
      context.artifacts['repository'], 
      return_dict=True, 
      load_in_4bit=True, 
      device_map={"":0},
      trust_remote_code=True,
    )
    self.model = PeftModel.from_pretrained(base_model, context.artifacts['lora'])
  
  def predict(self, context, model_input):
    prompt = model_input["prompt"][0]
    temperature = model_input.get("temperature", [1.0])[0]
    max_tokens = model_input.get("max_tokens", [100])[0]
    batch = self.tokenizer(prompt, padding=True, truncation=True,return_tensors='pt').to('cuda')
    with torch.cuda.amp.autocast():
      output_tokens = self.model.generate(
          input_ids = batch.input_ids, 
          max_new_tokens=max_tokens,
          temperature=temperature,
          top_p=0.7,
          num_return_sequences=1,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id,
          eos_token_id=tokenizer.eos_token_id,
      )
    generated_text = self.tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return generated_text

# COMMAND ----------

from mlflow.models.signature import ModelSignature
from mlflow.types import DataType, Schema, ColSpec
import pandas as pd
import mlflow

# Define input and output schema
input_schema = Schema([
    ColSpec(DataType.string, "prompt"), 
    ColSpec(DataType.double, "temperature"), 
    ColSpec(DataType.long, "max_tokens")])
output_schema = Schema([ColSpec(DataType.string)])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

# Define input example
input_example=pd.DataFrame({
            "prompt":["what is ML?"], 
            "temperature": [0.5],
            "max_tokens": [100]})

with mlflow.start_run() as run:  
    mlflow.pyfunc.log_model(
        "model",
        python_model=LLAMAQLORA(),
        artifacts={'repository' : snapshot_location, "lora": peft_model_id},
        pip_requirements=["torch", "transformers", "accelerate", "einops", "loralib", "bitsandbytes", "peft"],
        input_example=pd.DataFrame({"prompt":["what is ML?"], "temperature": [0.5],"max_tokens": [100]}),
        signature=signature
    )

# COMMAND ----------

# MAGIC %md
# MAGIC Run model inference with the model logged in MLFlow.

# COMMAND ----------

import mlflow
import pandas as pd


prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
if one get corona and you are self isolating and it is not severe, is there any meds that one can take?

### Response: """
# Load model as a PyFuncModel.
run_id = run.info.run_id
logged_model = f"runs:/{run_id}/model"

loaded_model = mlflow.pyfunc.load_model(logged_model)

text_example=pd.DataFrame({
            "prompt":[prompt], 
            "temperature": [0.5],
            "max_tokens": [100]})

# Predict on a Pandas DataFrame.
loaded_model.predict(text_example)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/noda7/miniconda3/envs/llamav2/bin')}
  warn(msg)
c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\Users\noda7\miniconda3\envs\llamav2 did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)


CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\noda7\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\peft\utils\other.py:122: FutureWarning: prepare_model_for_int8_training is dep

...

2023/09/06 01:14:36 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/09/06 01:15:34 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - loralib (current: uninstalled, required: loralib)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nif one get corona and you are self isolating and it is not severe, is there any meds that one can take?\n\n### Response: \nYes, you can take paracetamol for fever and pain.\n\n### End:\nHope this helps.\n\n### End of response\n\n### End of input\n\n### End of response\n\n### End of input\n\n### End of response\n\n### End of input\n\n### End of response\n\n### End of input\n\n### End of response\n\n### End of input\n'

## fine-tuningできた

fine-tuning modelにプロンプトを投げてみる。

In [2]:
run_id = run.info.run_id
print(run_id)

93b3ec6bcf2b4474b445bbe4ca02bb2f


In [3]:
import mlflow
import pandas as pd


prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
How can I make a business with LLM?

### Response: """
# Load model as a PyFuncModel.
run_id = run.info.run_id # 今回は93b3ec6bcf2b4474b445bbe4ca02bb2f
logged_model = f"runs:/{run_id}/model"

loaded_model = mlflow.pyfunc.load_model(logged_model)

text_example=pd.DataFrame({
            "prompt":[prompt], 
            "temperature": [0.5],
            "max_tokens": [100]})

# Predict on a Pandas DataFrame.
loaded_model.predict(text_example)

2023/09/06 01:31:59 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - loralib (current: uninstalled, required: loralib)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nHow can I make a business with LLM?\n\n### Response: \n1. Make a business that can generate a lot of data\n2. Use LLM to generate insights from the data\n\n### End:\nThis is just an idea.\n\n### End of response:\nThis is just an idea.\n\n### End of the response:\nThis is just an idea.\n\n### End of the response:\nThis is just an idea.\n\n### End of the response:\nThis is just an'

max_tokensを増やしてみる。

In [4]:
import mlflow
import pandas as pd


prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
How can I make a business with LLM?

### Response: """
# Load model as a PyFuncModel.
run_id = run.info.run_id # 今回は93b3ec6bcf2b4474b445bbe4ca02bb2f
logged_model = f"runs:/{run_id}/model"

loaded_model = mlflow.pyfunc.load_model(logged_model)

text_example=pd.DataFrame({
            "prompt":[prompt], 
            "temperature": [0.5],
            "max_tokens": [300]})

# Predict on a Pandas DataFrame.
loaded_model.predict(text_example)

2023/09/06 01:33:45 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - loralib (current: uninstalled, required: loralib)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nHow can I make a business with LLM?\n\n### Response: \nYou can make a business with LLM by using it to automate tasks and processes, such as customer service, data analysis, and legal research.\n\n### End:\nYou can make a business with LLM by using it to automate tasks and processes, such as customer service, data analysis, and legal research.\n\n### End:\nYou can make a business with LLM by using it to automate tasks and processes, such as customer service, data analysis, and legal research.\n\n### End:\nYou can make a business with LLM by using it to automate tasks and processes, such as customer service, data analysis, and legal research.\n\n### End:\nYou can make a business with LLM by using it to automate tasks and processes, such as customer service, data analysis, and legal research.\n\n### End:\nYou can make a business with LLM by using it to automate ta

同じ回答が繰り返されるようになった？

# 推論速度を向上させる
[Medium: GPTQ or bitsandbytes: Which Quantization Method to Use for LLMs — Examples with Llama 2](https://towardsdatascience.com/gptq-or-bitsandbytes-which-quantization-method-to-use-for-llms-examples-with-llama-2-f79bc03046dc)

If you are developing/deploying LLMs on consumer hardware, I would recommend the following:
* Fine-tune the LLM with bitsandbytes nf4 and QLoRa
* Merge the adapter into the LLM
* Quantize the resulting model with GPTQ 4-bit
* Deploy  